In [36]:
import csv
from typing import Dict, List
from llama_index import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import FaissVectorStore
from llama_index.prompts import PromptTemplate
from llama_index.llms import OpenAI, Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

## Global Variables
## Ollama LLM. This can be easily switched to GPT-4 or GPT-4o
LLAMA3 = Ollama(model="llama3",temperature=0.1)
# GPT4 = OpenAI(temperature=0.1, model="gpt-4")

In [34]:
from llama_index import embeddings

In [35]:
dir(embeddings)

['AdapterEmbeddingModel',
 'AzureOpenAIEmbedding',
 'BaseEmbedding',
 'BedrockEmbedding',
 'ClarifaiEmbedding',
 'ClipEmbedding',
 'CohereEmbedding',
 'DEFAULT_HUGGINGFACE_EMBEDDING_MODEL',
 'ElasticsearchEmbedding',
 'ElasticsearchEmbeddings',
 'GooglePaLMEmbedding',
 'GoogleUnivSentEncoderEmbedding',
 'GradientEmbedding',
 'HuggingFaceEmbedding',
 'HuggingFaceInferenceAPIEmbedding',
 'HuggingFaceInferenceAPIEmbeddings',
 'InstructorEmbedding',
 'LLMRailsEmbedding',
 'LLMRailsEmbeddings',
 'LangchainEmbedding',
 'LinearAdapterEmbeddingModel',
 'OpenAIEmbedding',
 'OptimumEmbedding',
 'Pooling',
 'SimilarityMode',
 'TextEmbeddingsInference',
 'VoyageEmbedding',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'adapter',
 'azure_openai',
 'base',
 'bedrock',
 'clarifai',
 'clip',
 'cohereai',
 'elasticsearch',
 'google',
 'google_palm',
 'gradient',
 'huggingface',
 'huggingface_optimum',
 'huggin

In [25]:
LLAMA3.complete("Mary had three little")

CompletionResponse(text='Lambs! The classic start to the traditional nursery rhyme "Mary Had a Little Lamb".', additional_kwargs={}, raw=None, delta=None)

In [9]:
!wget https://raw.githubusercontent.com/shubhamgupta-dat/MIMIC_2_OMOP/main/data/OMOP_Schema.csv -O ingestion_data/OMOP_Schema.csv
!wget https://raw.githubusercontent.com/shubhamgupta-dat/MIMIC_2_OMOP/main/data/MIMIC_to_OMOP_Mapping.csv -O ingestion_data/MIMIC_to_OMOP_Mapping.csv

--2024-07-21 13:28:35--  https://raw.githubusercontent.com/shubhamgupta-dat/MIMIC_2_OMOP/main/data/OMOP_Schema.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 307381 (300K) [text/plain]
Saving to: ‘ingestion_data/OMOP_Schema.csv’

ingestion_data/OMOP 100%[===================>] 300.18K  --.-KB/s    in 0.09s   

2024-07-21 13:28:36 (3.33 MB/s) - ‘ingestion_data/OMOP_Schema.csv’ saved [307381/307381]

--2024-07-21 13:28:36--  https://raw.githubusercontent.com/shubhamgupta-dat/MIMIC_2_OMOP/main/data/MIMIC_to_OMOP_Mapping.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request

In [20]:
import pandas as pd
df_history = pd.read_csv('ingestion_data/MIMIC_to_OMOP_Mapping.csv')
df_history.rename(columns={
'SRC_ENT': 'Source Table',
'SRC_ATT': 'Source Associated Table Column',
'TGT_ENT': 'Target Table',
'TGT_ATT': 'Target Associated Table Column'
},inplace=True)

In [21]:
[row.to_dict() for idx, row in df_history.sample(10).iterrows()]

[{'Source Table': 'CALLOUT',
  'Source Associated Table Column': 'SUBMIT_WARDID',
  'Target Table': 'VISIT_DETAIL',
  'Target Associated Table Column': 'care_site_id'},
 {'Source Table': 'PROCEDUREEVENTS_MV',
  'Source Associated Table Column': 'SUBJECT_ID',
  'Target Table': 'PROCEDURE_OCCURRENCE',
  'Target Associated Table Column': 'person_id'},
 {'Source Table': 'DATETIMEEVENTS',
  'Source Associated Table Column': 'STORETIME',
  'Target Table': nan,
  'Target Associated Table Column': nan},
 {'Source Table': 'ADMISSIONS',
  'Source Associated Table Column': 'EDOUTTIME',
  'Target Table': nan,
  'Target Associated Table Column': nan},
 {'Source Table': 'LABEVENTS',
  'Source Associated Table Column': 'VALUE',
  'Target Table': 'MEASUREMENT',
  'Target Associated Table Column': 'value_as_string'},
 {'Source Table': 'CHARTEVENTS',
  'Source Associated Table Column': 'RESULTSTATUS',
  'Target Table': nan,
  'Target Associated Table Column': nan},
 {'Source Table': 'PROCEDUREEVENTS_MV'

In [24]:
import pandas as pd
from typing import Dict, List
from llama_index import VectorStoreIndex, SimpleDirectoryReader, Document, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import SimpleVectorStore
from llama_index.prompts import PromptTemplate
from llama_index.llms import OpenAI, Ollama

# import faiss

# # dimensions of text-ada-embedding-002
# d = 4096
# faiss_index = faiss.IndexFlatL2(d)

class OMOPIndexer:
    def __init__(self,llm_client,service_context):
        self.index = None
        # self.llm = 
        self.llm = llm_client
        self.service_context = service_context
        # self.embed_model = OllamaEmbedding(
        #     model_name="llama3",
        #     base_url="http://localhost:11434",
        #     ollama_additional_kwargs={"mirostat": 0},
        # )
        # self.service_context = ServiceContext.from_defaults(
        #     llm=self.llm,
        #     embed_model=self.embed_model
        # )

    def ingest_historical_mappings(self, csv_file_path: str):
        df_history = pd.read_csv(csv_file_path)
        df_history.rename(columns={
        'SRC_ENT': 'Source Table',
        'SRC_ATT': 'Source Associated Table Column',
        'TGT_ENT': 'Target Table',
        'TGT_ATT': 'Target Associated Table Column'
        },inplace=True)
        documents = []
        for _, row in df_history.iterrows():
            content = f"Source Table: {row['Source Table']}, Source Column: {row['Source Associated Table Column']}, "
            content += f"Target Table: {row['Target Table']}, Target Column: {row['Target Associated Table Column']}"
            documents.append(Document(text=content))

        parser = SimpleNodeParser.from_defaults()
        nodes = parser.get_nodes_from_documents(documents)

        vector_store = SimpleVectorStore()
        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        self.index = VectorStoreIndex(
            nodes, 
            storage_context=storage_context,
            service_context=self.service_context
        )
        

    def ingest_omop_metadata(self, metadata_file_path: str):
        df_metadata = pd.read_csv(metadata_file_path)
        documents = []
        for _, row in df_metadata.iterrows():
            content = f"Table: {row['TableName']}, Column: {row['ColumnName']}, "
            content += f"Data Type: {row['ColumnType']}, Description: {row['ColumnDesc']}"
            documents.append(Document(text=content))
        parser = SimpleNodeParser.from_defaults()
        if self.index is None:
            self.index = VectorStoreIndex.from_documents(documents)
        else:
            self.index.insert_nodes(parser.get_nodes_from_documents(documents))

    def get_mapping_output(self, table_columns: Dict[str, List[str]]) -> str:
        query_engine = self.index.as_query_engine(
            llm=self.llm,
            similarity_top_k=5,
        )

        mapping_prompt = PromptTemplate(
            "Given the following OMOP tables and columns: {table_columns}\n"
            "Provide a mapping to the corresponding source tables and columns based on the historical mappings and OMOP metadata.\n"
            "Include explanations for each mapping decision."
        )

        critique_prompt = PromptTemplate(
            "You are a data mapping expert. Review the following mapping output and provide feedback:\n"
            "{mapping_output}\n"
            "Identify any potential issues, inconsistencies, or areas for improvement in the mapping."
        )

        improvement_prompt = PromptTemplate(
            "Based on the original mapping and the critique, provide an improved mapping output:\n"
            "Original mapping: {original_mapping}\n"
            "Critique: {critique}\n"
            "Improved mapping:"
        )

        # Initial mapping
        initial_response = query_engine.query(
            mapping_prompt.format(table_columns=table_columns)
        )
        initial_mapping = initial_response.response
        print(initial_mapping)

        # Self-critique
        critique_response = query_engine.query(
            critique_prompt.format(mapping_output=initial_mapping)
        )
        critique = critique_response.response
        print(critique)

        # Improved mapping
        improvement_response = query_engine.query(
            improvement_prompt.format(
                original_mapping=initial_mapping, critique=critique
            )
        )
        improved_mapping = improvement_response.response

        return improved_mapping


# Usage example
indexer = OMOPIndexer(llm_client=LLAMA3,service_context=service_context)
indexer.ingest_historical_mappings("ingestion_data/MIMIC_to_OMOP_Mapping.csv")
indexer.ingest_omop_metadata("ingestion_data/OMOP_Schema.csv")

table_columns = {
    "person": ["person_id", "birth_datetime", "gender_concept_id"],
    "condition_occurrence": [
        "condition_occurrence_id",
        "person_id",
        "condition_concept_id",
    ],
}

mapping_output = indexer.get_mapping_output(table_columns)
print(mapping_output)

ReadTimeout: timed out

In [39]:
csv_file_path = './ingestion_data/MIMIC_to_OMOP_Mapping.csv'
df_history = pd.read_csv(csv_file_path)
df_history.rename(columns={
'SRC_ENT': 'Source Table',
'SRC_ATT': 'Source Associated Table Column',
'TGT_ENT': 'Target Table',
'TGT_ATT': 'Target Associated Table Column'
},inplace=True)
documents = []
for _, row in df_history.iterrows():
    content = f"Source Table: {row['Source Table']}, Source Column: {row['Source Associated Table Column']}, "
    content += f"Target Table: {row['Target Table']}, Target Column: {row['Target Associated Table Column']}"
    documents.append(Document(text=content))

parser = SimpleNodeParser.from_defaults()
nodes = parser.get_nodes_from_documents(documents)

In [40]:
nodes[0]

TextNode(id_='72f12ff0-180f-4fe6-be69-8336c68c3274', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='f1ea5f34-7116-4a46-808d-e4e36d45ca02', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6edc3c2a2a759c6144c9647bd3d2e0b2c4b51624f73398867d9cbd3a4ae937f3'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='298de127-e636-4b6d-8262-af29718e8596', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='8f7142c79643413b6a6cbfa72c20873f91f44fc820540b7d8ca616a6990b717e')}, hash='6edc3c2a2a759c6144c9647bd3d2e0b2c4b51624f73398867d9cbd3a4ae937f3', text='Source Table: ADMISSIONS, Source Column: SUBJECT_ID, Target Table: VISIT_OCCURRENCE, Target Column: person_id', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [43]:
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text",
    base_url="http://localhost:11434",
)

# VectorStoreIndex(embed_model=oll)

ImportError: cannot import name 'global_handler' from 'llama_index.core' (/home/shubham/projects/open_source/dissertation/.conda/lib/python3.11/site-packages/llama_index/core/__init__.py)

In [1]:
from llama_index import embeddings 
dir(embeddings)

ImportError: cannot import name 'ChatResponseAsyncGen' from partially initialized module 'llama_index.llms' (most likely due to a circular import) (/home/shubham/projects/open_source/dissertation/.conda/lib/python3.11/site-packages/llama_index/llms/__init__.py)

In [10]:
import pandas as pd
from typing import Dict, List
from llama_index import VectorStoreIndex, SimpleDirectoryReader, Document, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores import FaissVectorStore
from llama_index.prompts import PromptTemplate
from llama_index.llms import OpenAI, Ollama
# from llama_index.embeddings.ollama import OllamaEmbedding

In [16]:
# !pip install transformers torch einops

In [17]:
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.llms import OpenAI, Ollama

## Global Variables
## Ollama LLM. This can be easily switched to GPT-4 or GPT-4o
LLAMA3 = Ollama(model="llama3",temperature=0.1)

In [ ]:
embed_model = HuggingFaceEmbedding(
    model_name="nomic-ai/nomic-embed-text-v1",
    trust_remote_code=True
    )

In [19]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    llm=LLAMA3,
    embed_model=embed_model,
)

from llama_index import set_global_service_context
set_global_service_context(service_context)

pytorch_model.bin:   0%|          | 0.00/547M [00:00<?, ?B/s]

<All keys matched successfully>


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [15]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(llm=None)

LLM is explicitly disabled. Using MockLLM.


ValueError: 
******
Could not load OpenAI embedding model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

Consider using embed_model='local'.
Visit our documentation for more embedding options: https://docs.llamaindex.ai/en/stable/module_guides/models/embeddings.html#modules
******

In [14]:
# !pip install transformers torch
vectore_store = VectorStoreIndex(nodes=[],embed_model=embed_model,llm=None)

ValueError: 
******
Could not load OpenAI model. If you intended to use OpenAI, please check your OPENAI_API_KEY.
Original error:
No API key found for OpenAI.
Please set either the OPENAI_API_KEY environment variable or openai.api_key prior to initialization.
API keys can be found or created at https://platform.openai.com/account/api-keys

To disable the LLM entirely, set llm=None.
******

In [12]:
from llama_index.llms.utils import resolve_llm

Init signature:
VectorStoreIndex(
    nodes: Optional[Sequence[llama_index.schema.BaseNode]] = None,
    index_struct: Optional[llama_index.data_structs.data_structs.IndexDict] = None,
    service_context: Optional[llama_index.service_context.ServiceContext] = None,
    storage_context: Optional[llama_index.storage.storage_context.StorageContext] = None,
    use_async: bool = False,
    store_nodes_override: bool = False,
    insert_batch_size: int = 2048,
    show_progress: bool = False,
    **kwargs: Any,
) -> None
Source:        
class VectorStoreIndex(BaseIndex[IndexDict]):
    """Vector Store Index.

    Args:
        use_async (bool): Whether to use asynchronous calls. Defaults to False.
        show_progress (bool): Whether to show tqdm progress bars. Defaults to False.
        store_nodes_override (bool): set to True to always store Node objects in index
            store and document store even if vector store keeps text. Defaults to False
    """

    index_struct_cls = Index